# FINM 33150 | Final Project Draft

Stephen Coley, Harrison Holt, Josh, Kyle Parran

Feb-24-2025

## The Task

The end goal here is to invent, simulate, and analyze a quantitative trading strategy.

Our initial ideas for a strategy include:


Once we determine a viable strategy we will need to ensure such a strategy has the following features:
- Investment in publicly traded (but not necessarily electronically traded) assets
- A "recipe" for evaluating attractiveness of potential trades that
    - depend on pre-specified classes of info
    - could be followed by anyone, given the recipe and data sources
- A "recipe" for investment sizes, and position entry and exit rules, that
    - may link position size to attractiveness
    - ensures risk stays within limits and capital remains bounded
    - makes reasonable assumptions about entry and exit liquidity

Elements of trading strategies include lead-lag relationships, market making, carry trades, pair trading, factor investment, model reversion, merger arbitrage, technical signals, etc.

Our strategy should have the following features:
- Significant dependence on at least one class of information that is not twitter, equity ohlc, equity VWAPs, or equity trading volumes, unelss at a relatively high frequency (600 secs or less)
- As of at least some times in your simulation, simultaneously holds at least 5 distinct assets (different currencies, commodities, option maturities/strikes etc.)
- Produces a total of 40 trades or more over your entire analysis/simulation period, without excessive clustering
- Leverage, with documented and reasonable capital and risk control assumptions


## Outline

### 1. Introduction

### 2. Data Assembly

### 3. Data Analysis

--------Next Phase--------

### 4. Strategy Implementation

### 6. Backtesting

### 7. Performance Analysis

### 8. References

## Introduction

## Setup

In [1]:
import pandas as pd
import numpy as np
import nasdaqdatalink as ndl
import quandl
import wrds
from dotenv import load_dotenv
import os
import datetime
from datetime import timedelta
from plotnine import ggplot, aes, geom_line, scale_color_manual, geom_hline, labs, theme, element_text, facet_wrap, geom_histogram, ggtitle, geom_boxplot, stat_function, geom_density
from scipy.stats import norm
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
import seaborn as sns
import warnings
from mpl_toolkits.mplot3d import Axes3D
import functools
from scipy.stats import zscore
from sklearn.decomposition import PCA
from functools import lru_cache
import pytz

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

load_dotenv()

ndl.ApiConfig.api_key = os.getenv('NDL_API_KEY')
quandl_api_key = os.getenv('NDL_API_KEY')
wrds_username = os.getenv('WRDS_USERNAME')
adv_api_key = os.getenv('ADV_API_KEY')


## Data Assembly

### Quandl function

In [2]:
def grab_quandl_table(
    table_path,
    avoid_download=False,
    replace_existing=False,
    date_override=None,
    allow_old_file=False,
    **kwargs,
):
    root_data_dir = os.path.join(os.path.expanduser("~"), "quandl_data_table_downloads")
    if not os.path.exists(root_data_dir):
        print(f"Directory does not exist. Creating: {root_data_dir}")
        os.makedirs(root_data_dir, exist_ok=True)
    else:
        print(f"Directory already exists: {root_data_dir}")

    data_symlink = os.path.normpath(os.path.join(root_data_dir, f"{table_path}_latest.zip"))

    if avoid_download and os.path.exists(data_symlink):
        print(f"Skipping any possible download of {table_path}")
        return data_symlink
    
    table_dir = os.path.dirname(data_symlink)

    print(table_dir)
    if not os.path.isdir(table_dir):
        print(f'Creating new data dir {table_dir}')
        os.mkdir(table_dir)

    if date_override is None:
        my_date = datetime.datetime.now().strftime("%Y%m%d")
    else:
        my_date = date_override
    data_file = os.path.normpath(os.path.join(root_data_dir, f"{table_path}_{my_date}.zip"))

    if os.path.exists(data_file):
        file_size = os.stat(data_file).st_size
        if replace_existing or not file_size > 0:
            print(f"Removing old file {data_file} size {file_size}")
        else:
            print(
                f"Data file {data_file} size {file_size} exists already, no need to download"
            )
            return data_file

    dl = quandl.export_table(
        table_path, 
        filename=data_file, 
        api_key=os.getenv('NDL_API_KEY'), 
        **kwargs
    )
    file_size = os.stat(data_file).st_size
    if os.path.exists(data_file) and file_size > 0:
        print(f"Download finished: {file_size} bytes")
        print(data_symlink)
        if not date_override:
            if os.path.exists(data_symlink):
                print(f"Removing old symlink")
                os.unlink(data_symlink)
            print(f"Creating symlink: {data_file} -> {data_symlink}")
            os.symlink(
                data_file, data_symlink,
            )
    else:
        print(f"Data file {data_file} failed download")
        return
    return data_symlink if (date_override is None or allow_old_file) else "NoFileAvailable"


def fetch_quandl_table(table_path, avoid_download=True, **kwargs):
    return pd.read_csv(
        grab_quandl_table(table_path, avoid_download=avoid_download, **kwargs)
    )

### Data load

**Quandl EOD price data**

When referencing price end of day price data we will utilize Quandl End of Day US Stock Prices. Updated daily, this data feed offers end of day prices, dividends, adjustments and splits for U.S. publicly traded stocks with history to 1996. Prices are provided both adjusted and unadjusted.

Key Features:

- Covers all stocks with primary listing on NASDAQ, AMEX, NYSE and ARCA.
- Includes unadjusted and adjusted open, high, low, close, volume.
- Includes dividend history and split history.
- Updated at or before 5:00 pm ET on all trading days.
- Exchange corrections are applied by 9:30 pm ET.
- Historical data goes back to 1996.

In [ ]:
start_filing_date = '2014-01-01'
end_filing_date = '2024-12-31'
start_filter_date = '2019-01-01'
end_filter_date = '2024-12-31'

eod_data = fetch_quandl_table(
    'QUOTEMEDIA/PRICES', 
    avoid_download=True,
    date={'gte': f'{start_filing_date}','lte': f'{end_filing_date}'},
    qopts={'columns': ['ticker','date','adj_close', ]}
)

eod_data['date'] = pd.to_datetime(eod_data['date'], errors='coerce')
eod_data.dropna(subset=['date'], inplace=True)

Directory already exists: C:\Users\kylep\quandl_data_table_downloads
C:\Users\kylep\quandl_data_table_downloads\QUOTEMEDIA
Download finished: 237362542 bytes
C:\Users\kylep\quandl_data_table_downloads\QUOTEMEDIA\PRICES_latest.zip
Creating symlink: C:\Users\kylep\quandl_data_table_downloads\QUOTEMEDIA\PRICES_20250301.zip -> C:\Users\kylep\quandl_data_table_downloads\QUOTEMEDIA\PRICES_latest.zip


In [4]:
eod_data.head()

,ticker,date,open,high,low,close,volume,dividend,split,adj_open,adj_high,adj_low,adj_close,adj_volume
0,JTKWY,2022-03-11,6.1700,7.3200,5.7900,6.72,9440097.0,0.0,1.0,6.1700,7.3200,5.7900,6.72,9440097.0
1,JTKWY,2022-03-10,6.1600,6.1750,5.9350,6.07,2261623.0,0.0,1.0,6.1600,6.1750,5.9350,6.07,2261623.0
2,FG_1,2020-06-01,8.1000,8.3900,8.1000,8.39,3086317.0,0.0,1.0,8.1000,8.3900,8.1000,8.39,3086317.0
3,YTENQ,2024-09-30,0.9514,1.0500,0.9514,1.05,842.0,0.0,1.0,0.9514,1.0500,0.9514,1.05,842.0
4,FLWS,2022-03-09,14.5700,14.9588,14.4100,14.45,662492.0,0.0,1.0,14.5700,14.9588,14.4100,14.45,662492.0


**Zacks Fundamental B**

| Table                       | Table Code   | Table Description                                                                                                                                                                                                                                                                   |
|-----------------------------|--------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Fundamentals Condensed      | ZACKS/FC     | This table contains over 200 fundamental indicators for over 19,500 companies including over 10,000 delisted stocks. For access to additional indicators see Zacks Fundamentals C.                                                                                                    |
| Fundamentals Ratios         | ZACKS/FR     | This table contains 26 fundamental ratios for over 19,500 companies, including over 10,000 delisted stocks.                                                                                                                                                                           |
| Master Table                | ZACKS/MT     | This table contains descriptive information about all tickers that are included in Zacks products.                                                                                                                                                                                   |
| Market Value Supplement     | ZACKS/MKTV   | This table contains supplementary information to Zacks fundamentals, with updated values for market capitalization and enterprise value. Market cap is the total value of all of a company's outstanding stock. You calculate it by multiplying the current stock price by the number of shares owned by stockholders. |
| Shares Out Supplement       | ZACKS/SHRS   | This table contains supplementary information to Zacks fundamentals, with updated values for common shares outstanding and average diluted shares outstanding. Shares outstanding are the total number of shares issued by a corporation.                                              |
| Historical Daily Maintenance| ZACKS/HDM    | This table offers historical daily corporate actions for over 7,000 U.S. and Canadian listed companies. Sometimes the price of a stock changes because of events that are not associated with the supply-and-demand forces of the markets. For instance, if a company declares a stock split or reverse stock split, the share price will change radically, even though it does not actually affect the value of the company or investors' holdings in the stock. The adjusted close price shows the effect of these events. |

In [5]:


year_beginning_filing = pd.to_datetime(start_filing_date).year
year_end_filing = pd.to_datetime(end_filing_date).year

year_beginning_filter = pd.to_datetime(start_filter_date).year
year_end_filter = pd.to_datetime(end_filter_date).year

zacks_fc = fetch_quandl_table('ZACKS/FC', per_end_date = {'gte':start_filing_date, 'lte':end_filing_date}, avoid_download=True)
zacks_fr = fetch_quandl_table('ZACKS/FR', per_end_date = {'gte':start_filing_date, 'lte':end_filing_date}, avoid_download=True)
zacks_mt = fetch_quandl_table('ZACKS/MT', avoid_download=True)
zacks_mktv = fetch_quandl_table('ZACKS/MKTV', per_end_date = {'gte':start_filing_date, 'lte':end_filing_date}, avoid_download=True)
zacks_shrs = fetch_quandl_table('ZACKS/SHRS', per_end_date = {'gte':start_filing_date, 'lte':end_filing_date}, avoid_download=True)
zacks_hdm = fetch_quandl_table('ZACKS/HDM', avoid_download=True)

zacks_fc['per_end_date'] = pd.to_datetime(zacks_fc['per_end_date'])
zacks_fc['filing_date'] = pd.to_datetime(zacks_fc['filing_date'])+timedelta(days=1)
zacks_fr['per_end_date'] = pd.to_datetime(zacks_fr['per_end_date'])
zacks_mktv['per_end_date'] = pd.to_datetime(zacks_mktv['per_end_date'])
zacks_shrs['per_end_date'] = pd.to_datetime(zacks_shrs['per_end_date'])
zacks_fc = zacks_fc.dropna(subset=['per_end_date', 'filing_date'])

zacks_df = pd.merge(zacks_fc, zacks_fr, on=['ticker', 'per_end_date', 'per_type'], how='left', suffixes=('_fc', '_fr'))
zacks_df = pd.merge(zacks_df, zacks_mktv, on=['ticker', 'per_end_date', 'per_type'], how='left')
zacks_df = pd.merge(zacks_df, zacks_shrs, on=['ticker', 'per_end_date', 'per_type'], how='left')
zacks_df = pd.merge(zacks_df, zacks_mt, on=['ticker'], how='left')

Directory already exists: C:\Users\kylep\quandl_data_table_downloads
Skipping any possible download of ZACKS/FC
Directory already exists: C:\Users\kylep\quandl_data_table_downloads
Skipping any possible download of ZACKS/FR
Directory already exists: C:\Users\kylep\quandl_data_table_downloads
Skipping any possible download of ZACKS/MT
Directory already exists: C:\Users\kylep\quandl_data_table_downloads
Skipping any possible download of ZACKS/MKTV
Directory already exists: C:\Users\kylep\quandl_data_table_downloads
Skipping any possible download of ZACKS/SHRS
Directory already exists: C:\Users\kylep\quandl_data_table_downloads
Skipping any possible download of ZACKS/HDM


**WRDS Market Data Bars (WRDS TAQ)**

WRDS TAQ has millisecond-level best bid and offer data.  However, downloading the full data series is not practical due to the hundreds of millions of rows per day.  Here, we use SQL to generate more manageable "bars" on the server side.

In [ ]:
@lru_cache
def taq_nbbo_bars_on_date(tickers: list[str] | str, date:datetime.date, bar_minutes=30):
    assert bar_minutes==60 or (bar_minutes<=30 and 30%bar_minutes==0)
    date_str = date.strftime('%Y%m%d')
    year_str = date.strftime('%Y')
    tickers = (tickers,) if hasattr(tickers,'strip') else tuple(tickers)  # Allow single ticker as argument

    sql = f"""
            WITH windowable_nbbo AS (
                SELECT
                    sym_root AS ticker
                    , date
                    , time_m
                    , time_m_nano
                    , sym_root
                    , qu_cond
                    , best_bid
                    , best_bidsizeshares
                    , best_ask
                    , best_asksizeshares
                    , EXTRACT(HOUR FROM time_m) AS hour_of_day
                    , {bar_minutes} * DIV(EXTRACT(MINUTE FROM time_m),{bar_minutes}) AS minute_of_hour
                    , ROW_NUMBER() OVER (PARTITION BY sym_root, EXTRACT(HOUR FROM time_m), DIV(EXTRACT(MINUTE FROM time_m),{bar_minutes}) ORDER BY time_m DESC) AS rownum
                FROM taqm_{year_str}.complete_nbbo_{date_str} 
                WHERE 1=1
                  AND sym_root IN {tickers}
                  AND sym_suffix IS NULL
                  AND time_m > '09:30:00' AND time_m < '16:00:00'
            )
            SELECT DISTINCT ON (ticker, date, hour_of_day, minute_of_hour)
                ticker
                , date
                , date + (hour_of_day || ':' || minute_of_hour)::interval + ( '00:{bar_minutes}' )::interval AS window_time
                , best_bid
                , best_bidsizeshares
                , best_ask
                , best_asksizeshares
                , time_m AS time_of_last_quote
                , time_m_nano AS time_of_last_quote_ns
            FROM windowable_nbbo
            WHERE windowable_nbbo.rownum = 1
            """
    bars = db.raw_sql(sql)
    
    def _make_timestamp(r):
        t = datetime.datetime.combine(r.date, r.time_of_last_quote)#, tzinfo=pytz.timezone('America/New_York'))
        pdt = pd.to_datetime(t).tz_localize(pytz.timezone('America/New_York')) + pd.Timedelta(r.time_of_last_quote_ns, unit="ns")
        return pdt
    bars['time_of_last_quote'] = bars.apply(_make_timestamp, axis=1)
    del bars['time_of_last_quote_ns']
    bars['window_time'] = pd.to_datetime(bars['window_time']).dt.tz_localize(pytz.timezone('America/New_York'))
    return bars

In [ ]:
b20240229 = taq_nbbo_bars_on_date(('SPY', 'PBPB', 'HLIT'), date=datetime.date(2024,2,29))
b20240229.set_index(['ticker', 'date', 'window_time']).head()

**WRDS I/B/E/S Summary History**

​Refinitiv's Institutional Brokers' Estimate System (I/B/E/S) offers a comprehensive database of analyst forecasts and actual earnings data for publicly traded companies worldwide. The "Surprise History" within I/B/E/S specifically focuses on earnings surprises, providing insights into how actual earnings compare to analyst expectations over time.

Key components of the I/B/E/S Surprise History data include:
- Analyst Estimates: Forecasts for various financial metrics, notably Earnings Per Share (EPS), collected from a vast network of analysts. These estimates are aggregated to form consensus figures, serving as benchmarks for actual performance.​
- Actual Reported Earnings: The realized financial figures reported by companies, allowing for direct comparison against prior analyst projections.​




In [6]:
import wrds
db = wrds.Connection()
surp_sum_query = f"""
SELECT *
FROM tr_ibes.surpsum
WHERE measure = 'EPS'
  AND fiscalp = 'QTR'
  AND oftic IS NOT NULL
  AND anndats >= '{start_filing_date}'
  AND anndats <= '{end_filing_date}'
"""
surp_sum = db.raw_sql(surp_sum_query)
surp_sum.head()


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


,ticker,oftic,measure,fiscalp,pyear,pmon,usfirm,anndats,actual,surpmean,surpstdev,suescore
0,0000,TLMR,EPS,QTR,2014.0,3.0,1,2014-05-06,0.12,0.08000,0.01414,2.82845
1,0000,TLMR,EPS,QTR,2014.0,6.0,1,2014-08-06,0.27,0.13000,0.01225,11.43137
2,0000,TLMR,EPS,QTR,2014.0,9.0,1,2014-11-04,0.26,0.23200,0.05848,0.47879
3,0000,TLMR,EPS,QTR,2014.0,12.0,1,2015-01-30,0.16,0.17167,0.01169,-0.99800
4,0000,TLMR,EPS,QTR,2015.0,3.0,1,2015-04-30,0.12,0.17833,0.01835,-3.17927


**Fama-French Factor Returns**

We obtained the Fama-French factor returns (SMB, HML, RF and Mkt-RF) over the same data period as trading period (reference https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html).

In [55]:
ff_file_path = rf"../data/F-F_Research_Data_Factors_daily.csv"
ff_data = pd.read_csv(ff_file_path)
ff_data['date'] = pd.to_datetime(ff_data['date'], format='%Y%m%d')
ff_data.head()

,date,Mkt-RF,SMB,HML,RF
0,1926-07-01,0.10,-0.25,-0.27,0.009
1,1926-07-02,0.45,-0.33,-0.06,0.009
2,1926-07-06,0.17,0.30,-0.39,0.009
3,1926-07-07,0.09,-0.58,0.02,0.009
4,1926-07-08,0.21,-0.38,0.19,0.009


### Data Cleansing/Merge

In [7]:
# Cleaning the surprise Data
surp_sum['anndats'] = pd.to_datetime(surp_sum['anndats'], errors='coerce')
surp_sum = surp_sum[surp_sum['anndats'].notnull()]

surp_sum['year'] = surp_sum['anndats'].dt.year

years_required = set(range(year_beginning_filter, year_end_filter+1))

coverage = surp_sum.groupby('oftic')['year'].apply(set)
keep_tickers = coverage[coverage.apply(lambda x: years_required.issubset(x))].index
surp_sum = surp_sum[surp_sum['oftic'].isin(keep_tickers)]

quarter_counts = (
    surp_sum
    .groupby(['oftic', 'year'])
    .size()
    .reset_index(name='count_per_year')
)

quarter_counts_ok = quarter_counts[quarter_counts['count_per_year'] >= 4]

annual_coverage = quarter_counts_ok.groupby('oftic')['year'].apply(set)
keep_tickers_4q = annual_coverage[annual_coverage.apply(lambda x: years_required.issubset(x))].index

# Filtering out stocks that dont have 4 quarters of data for each year
surp_sum = surp_sum[surp_sum['oftic'].isin(keep_tickers_4q)]
surp_sum['oftic'].nunique()

3632

In [8]:
# Setting The Market Cap Threshold to Filter out Smaller Companies
mkt_cap_threshold = 500

# Selecting the metrics we plan to include in our strategy
zacks_df = zacks_df.copy()
zacks_df = zacks_df[zacks_df['per_end_date'].notnull() & (zacks_df['per_type'] == 'Q')]
zacks_df.dropna(subset=['tot_revnu','ebitda','eps_basic_net','mkt_val','ep_val','shares_out'], inplace=True)
zacks_df.sort_values(['ticker','per_end_date'], inplace=True)

# Creating Other metrics we plan to use as indicators
zacks_df = zacks_df[zacks_df['per_end_date'].between(start_filing_date, end_filing_date)]
zacks_df['price'] = zacks_df['mkt_val'] / zacks_df['shares_out']
zacks_df['PE'] = zacks_df['price'] / zacks_df['eps_basic_net']
zacks_df['EV_EBITDA'] = zacks_df['ep_val'] / zacks_df['ebitda']
zacks_df['rev_yoy_growth'] = zacks_df.groupby('ticker')['tot_revnu'].pct_change(periods=4)
zacks_df['price_yoy_growth'] = zacks_df.groupby('ticker')['price'].pct_change(periods=4)

# Filtering Stocks to a smaller date range
zacks_df = zacks_df[zacks_df['per_end_date'].between(start_filter_date, end_filter_date)]
zacks_df['year'] = zacks_df['per_end_date'].dt.year

# Filtering out stocks with Market Cap below the threshold
min_cap_by_ticker = zacks_df.groupby('ticker')['mkt_val'].transform('min')
zacks_df = zacks_df[min_cap_by_ticker >= mkt_cap_threshold]

years_required = set(range(year_beginning_filter, year_end_filter + 1))
coverage = zacks_df.groupby('ticker')['year'].apply(set)
keep_tickers = coverage[coverage.apply(lambda s: years_required.issubset(s))].index
zacks_df = zacks_df[zacks_df['ticker'].isin(keep_tickers)]

counts = zacks_df.groupby(['ticker', 'year']).size().reset_index(name='count_per_year')
counts_ok = counts[counts['count_per_year'] >= 4]
annual_coverage = counts_ok.groupby('ticker')['year'].apply(set)
keep_tickers_4q = annual_coverage[annual_coverage.apply(lambda s: years_required.issubset(s))].index
zacks_df = zacks_df[zacks_df['ticker'].isin(keep_tickers_4q)]
zacks_df['ticker'].nunique()

374

In [9]:
# Filtering the surprise data on market cap requirement
valid_tickers = set(zacks_df['ticker'].unique())
surp_sum_filtered = surp_sum[surp_sum['oftic'].isin(valid_tickers)]
surp_sum_filtered['oftic'].nunique()


327

In [13]:
surp_sum_filtered.head()

,ticker,oftic,measure,fiscalp,pyear,pmon,usfirm,anndats,actual,surpmean,surpstdev,suescore,year
274,001J,KN,EPS,QTR,2014.0,3.0,1,2014-04-28,0.32,0.30333,0.01155,1.44338,2014
275,001J,KN,EPS,QTR,2014.0,6.0,1,2014-07-28,0.21,0.17600,0.00894,3.80143,2014
276,001J,KN,EPS,QTR,2014.0,9.0,1,2014-10-27,0.38,0.40500,0.05505,-0.45417,2014
277,001J,KN,EPS,QTR,2014.0,12.0,1,2015-02-12,0.14,0.34125,0.02232,-9.01617,2015
278,001J,KN,EPS,QTR,2015.0,3.0,1,2015-04-23,0.06,0.02750,0.01832,1.77373,2015


In [ ]:
# Separating the Surprises into Bins
surp_sum_filtered['avg_eps'] = surp_sum_filtered.groupby('ticker')['actual'].transform('mean')
bins = [-np.inf, 0, 0.5, 1, 2,  np.inf]
labels = [
    "Negative",
    "0 to 0.5",
    "0.5 to 1.0",
    "1.0 to 2.0",
    ">2.0"
]

surp_sum_filtered['avg_eps_bin'] = pd.cut(surp_sum_filtered['avg_eps'], bins=bins, labels=labels)


surp_sum_filtered.groupby('avg_eps_bin')['ticker'].nunique()

avg_eps_bin
Negative       33
0 to 0.5      126
0.5 to 1.0    122
1.0 to 2.0     85
>2.0           47
Name: ticker, dtype: int64

In [ ]:
# Sorting close prices for merge
eod_data.sort_values(['ticker','date'], inplace=True)

# Creating the next day close price and next day return
eod_data['next_close'] = eod_data.groupby('ticker')['adj_close'].shift(-1)
eod_data['next_day_return'] = (
    eod_data['next_close'] / eod_data['adj_close'] - 1
)
# Merging the surprise data with the EOD data
surp_sum_filtered = surp_sum_filtered.merge(
    eod_data[['ticker','date','adj_close','next_close','next_day_return']],
    how='left',
    left_on=['oftic','anndats'],
    right_on=['ticker','date']
)

In [25]:
surp_sum_filtered.head()

,ticker_x,oftic,measure,fiscalp,pyear,pmon,usfirm,anndats,actual,surpmean,surpstdev,suescore,year,eps_bin,avg_eps,avg_eps_bin,ticker_y,date,adj_close,next_close,next_day_return
0,001J,KN,EPS,QTR,2014.0,3.0,1,2014-04-28,0.32,0.30333,0.01155,1.44338,2014,0 to 0.5,0.243571,0 to 0.5,KN,2014-04-28,30.92,29.52,-0.045278
1,001J,KN,EPS,QTR,2014.0,6.0,1,2014-07-28,0.21,0.17600,0.00894,3.80143,2014,0 to 0.5,0.243571,0 to 0.5,KN,2014-07-28,29.08,28.69,-0.013411
2,001J,KN,EPS,QTR,2014.0,9.0,1,2014-10-27,0.38,0.40500,0.05505,-0.45417,2014,0 to 0.5,0.243571,0 to 0.5,KN,2014-10-27,18.80,19.40,0.031915
3,001J,KN,EPS,QTR,2014.0,12.0,1,2015-02-12,0.14,0.34125,0.02232,-9.01617,2015,0 to 0.5,0.243571,0 to 0.5,KN,2015-02-12,21.64,18.59,-0.140943
4,001J,KN,EPS,QTR,2015.0,3.0,1,2015-04-23,0.06,0.02750,0.01832,1.77373,2015,0 to 0.5,0.243571,0 to 0.5,KN,2015-04-23,21.24,20.08,-0.054614


In [38]:
# Calculating avg_estimate
surp_sum_filtered['eps_estimate'] = surp_sum_filtered['actual'] - surp_sum_filtered['surpmean']

# Calculating percent surprise
surp_sum_filtered['pct_surprise'] = (
    surp_sum_filtered['surpmean'] / surp_sum_filtered['eps_estimate']
)

# Creating a metric to estimate the drift available
surp_sum_filtered['surprise_vs_move'] = (
    surp_sum_filtered['pct_surprise'] / surp_sum_filtered['next_day_return']
)
surp_sum_filtered['surprise_vs_move'] = surp_sum_filtered['surprise_vs_move'].replace([np.inf, -np.inf], 0)


In [39]:
surp_sum_filtered['zscore_surprise_vs_move'] = (
    surp_sum_filtered
    .groupby('avg_eps_bin')['surprise_vs_move']
    .transform(lambda x: (x - x.mean()) / x.std())
)
surp_sum_filtered.head()


,ticker_x,oftic,measure,fiscalp,pyear,pmon,usfirm,anndats,actual,surpmean,surpstdev,suescore,year,eps_bin,avg_eps,avg_eps_bin,ticker_y,date,adj_close,next_close,next_day_return,eps_estimate,pct_surprise,surprise_vs_move,zscore_surprise_vs_move
0,001J,KN,EPS,QTR,2014.0,3.0,1,2014-04-28,0.32,0.30333,0.01155,1.44338,2014,0 to 0.5,0.243571,0 to 0.5,KN,2014-04-28,30.92,29.52,-0.045278,0.01667,18.196161,-401.875208,-0.025239
1,001J,KN,EPS,QTR,2014.0,6.0,1,2014-07-28,0.21,0.17600,0.00894,3.80143,2014,0 to 0.5,0.243571,0 to 0.5,KN,2014-07-28,29.08,28.69,-0.013411,0.03400,5.176471,-385.978884,-0.024862
2,001J,KN,EPS,QTR,2014.0,9.0,1,2014-10-27,0.38,0.40500,0.05505,-0.45417,2014,0 to 0.5,0.243571,0 to 0.5,KN,2014-10-27,18.80,19.40,0.031915,-0.02500,-16.200000,-507.600000,-0.027743
3,001J,KN,EPS,QTR,2014.0,12.0,1,2015-02-12,0.14,0.34125,0.02232,-9.01617,2015,0 to 0.5,0.243571,0 to 0.5,KN,2015-02-12,21.64,18.59,-0.140943,-0.20125,-1.695652,12.030791,-0.015435
4,001J,KN,EPS,QTR,2015.0,3.0,1,2015-04-23,0.06,0.02750,0.01832,1.77373,2015,0 to 0.5,0.243571,0 to 0.5,KN,2015-04-23,21.24,20.08,-0.054614,0.03250,0.846154,-15.493369,-0.016087


## Data Analysis

## Strategy Implementation

## Backtesting

## Performance Analysis

## References